In [5]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma

from langchain_community.embeddings import HuggingFaceBgeEmbeddings

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.llms import HuggingFaceHub


from langchain.chains import RetrievalQA

loaders = [
    PyPDFLoader("data/cocofoods.pdf"), 
]

docs = []
chunk_size=150
chunk_overlap = 50

for loader in loaders:
    docs.extend(loader.load())

model_name = 'BAAI/bge-small-en'
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings':True}


r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
    )

embedding = HuggingFaceBgeEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
    )

llm = HuggingFaceHub(
    repo_id = 'google/flan-t5-xxl',
    model_kwargs = {'temperature':0,'max_length':500},
    huggingfacehub_api_token = ""
    )

splits = r_splitter.split_documents(docs)


In [6]:
def ans_this(question,splits,embedding,llm):
    persist_directory = 'docs/chroma'

    vectordb = Chroma.from_documents(
        documents = splits,
        embedding = embedding,
        persist_directory = persist_directory)

    docs = vectordb.similarity_search(f'{question}',k=3)
    
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever = vectordb.as_retriever())

    result = qa_chain({'query':f'{question}'})

    return result['result']

In [12]:
print("CHAT BOT: Hi ask me any question about the uploaded document\n")
while(True):
    
    print("USER:",end='')
    question = input()
    
    if(question==''):
        break
    answer = ans_this(question,splits,embedding,llm)
    
    print("CHAT BOT: ",answer,'\n')
    
    
    #To exit press enter

CHAT BOT: Hi ask me any question about the uploaded document

USER:What is name of restaurant
CHAT BOT:  CocoFoods 

USER:Who is the owner of restaurant
CHAT BOT:  Harris Morsion 

USER:What are the best dishes of the restaurant
CHAT BOT:  Oreo Fudge Cake 

USER:What is the location of restaurant
CHAT BOT:  I don't know 

USER:What is its location
CHAT BOT:  455, Bekshire Wall Streets 

USER:what is the website of restaurant
CHAT BOT:  cocofoods.com 

USER:
